# Demonstration of Noise

In [1]:
import os
import sys
import numpy as np
import torch
from PIL import Image
from torchvision import transforms

# Add the parent directory to the Python path so we can import from the modules folder
sys.path.append(os.path.abspath(".."))

# Import custom modules for preprocessing and embedding
from modules.extraction.preprocessing import Preprocessing
from modules.extraction.embedding import Embedding

# ----------------------------
# Helper Functions
# ----------------------------
def load_first_image(directory):
    """
    Load the first valid image file found in the given directory,
    filtering out hidden/system files.
    """
    files = [f for f in os.listdir(directory)
             if f.lower().endswith(('.jpg', '.png')) and not f.startswith("._")]
    if not files:
        raise FileNotFoundError(f"No image files found in {directory}")
    files.sort()  # Ensure a consistent order
    return os.path.join(directory, files[0])

def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

def dot_product_distance(vec1, vec2):
    return -np.dot(vec1, vec2)

def cosine_distance(vec1, vec2):
    cos_sim = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return 1 - cos_sim

def minkowski_distance(vec1, vec2, p=3):
    return np.sum(np.abs(vec1 - vec2) ** p) ** (1/p)

# ----------------------------
# Define Global Variables and Paths
# ----------------------------
# List of individuals (probe identities)
individuals = ['Drew_Barrymore', 'Warren_Buffett', 'Owen_Wilson', 'Nelson_Mandela', 'Ian_Thorpe']

# Get the parent directory (assumes the structure: parent_dir/storage/...)
parent_dir = os.path.abspath("..")

# Build dictionaries of file paths for probe and gallery images.
probe_images = {}
gallery_images = {}
for person in individuals:
    probe_dir = os.path.join(parent_dir, "storage", "probe", person)
    gallery_dir = os.path.join(parent_dir, "storage", "multi_image_gallery", person)
    probe_images[person] = load_first_image(probe_dir)
    gallery_images[person] = load_first_image(gallery_dir)

print("Probe Images:")
for person, path in probe_images.items():
    print(f"  {person}: {path}")

print("\nGallery Images:")
for person, path in gallery_images.items():
    print(f"  {person}: {path}")

# ----------------------------
# Initialize Preprocessing and Embedding
# ----------------------------
preprocessor = Preprocessing(image_size=160)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Initialize the embedding model (using casia-webface for gallery precomputation)
embedding_casia = Embedding(pretrained='casia-webface', device=device)

# ----------------------------
# Precompute Gallery Embeddings (Task 2)
# ----------------------------
gallery_embeddings = []
gallery_base = os.path.join(parent_dir, "storage", "multi_image_gallery")
# List all person directories in the gallery
persons_in_gallery = [d for d in os.listdir(gallery_base) if os.path.isdir(os.path.join(gallery_base, d))]

for person in persons_in_gallery:
    person_dir = os.path.join(gallery_base, person)
    # Get all valid image files (filter out hidden/system files)
    image_files = [f for f in os.listdir(person_dir)
                   if f.lower().endswith(('.jpg', '.png')) and not f.startswith("._")]
    image_files.sort()
    for img in image_files:
        img_path = os.path.join(person_dir, img)
        try:
            image = Image.open(img_path)
            tensor = preprocessor.process(image)
            embedding = embedding_casia.encode(tensor)
            gallery_embeddings.append({"person": person, "image_path": img_path, "embedding": embedding})
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

print(f"\nPrecomputed embeddings for {len(gallery_embeddings)} gallery images.")


Probe Images:
  Drew_Barrymore: C:\Users\Putna\OneDrive - Johns Hopkins\Documents\Johns Hopkins\Creating AI Enabled Systems\SP25\ironclad\storage\probe\Drew_Barrymore\Drew_Barrymore_0002.jpg
  Warren_Buffett: C:\Users\Putna\OneDrive - Johns Hopkins\Documents\Johns Hopkins\Creating AI Enabled Systems\SP25\ironclad\storage\probe\Warren_Buffett\Warren_Buffett_0002.jpg
  Owen_Wilson: C:\Users\Putna\OneDrive - Johns Hopkins\Documents\Johns Hopkins\Creating AI Enabled Systems\SP25\ironclad\storage\probe\Owen_Wilson\Owen_Wilson_0002.jpg
  Nelson_Mandela: C:\Users\Putna\OneDrive - Johns Hopkins\Documents\Johns Hopkins\Creating AI Enabled Systems\SP25\ironclad\storage\probe\Nelson_Mandela\Nelson_Mandela_0002.jpg
  Ian_Thorpe: C:\Users\Putna\OneDrive - Johns Hopkins\Documents\Johns Hopkins\Creating AI Enabled Systems\SP25\ironclad\storage\probe\Ian_Thorpe\Ian_Thorpe_0002.jpg

Gallery Images:
  Drew_Barrymore: C:\Users\Putna\OneDrive - Johns Hopkins\Documents\Johns Hopkins\Creating AI Enabled Sys

In [2]:
from torchvision import transforms

# Define transformation pipelines for probe images.
# Note: We apply these transforms to the PIL image BEFORE preprocessing.
transformations = {
    "horizontal_flip": transforms.RandomHorizontalFlip(p=1.0),
    "gaussian_blur": transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
    "increase_brightness": transforms.ColorJitter(brightness=2.0),
    "decrease_brightness": transforms.ColorJitter(brightness=0.5)
}

print("\n--- Rank Positions for Correct Gallery Matches on Transformed Probe Images ---\n")

# Loop through each transformation and process each probe image.
for transform_name, transform_func in transformations.items():
    print(f"\nTransformation: {transform_name}")
    for person in individuals:
        # Open the original probe image.
        image = Image.open(probe_images[person])
        # Apply the transformation.
        transformed_image = transform_func(image)
        
        # Preprocess and compute the embedding for the transformed probe image.
        tensor = preprocessor.process(transformed_image)
        probe_embedding = embedding_casia.encode(tensor)
        
        # Compute distances between the transformed probe embedding and each gallery embedding.
        distances = {"euclidean": [], "dot_product": [], "cosine": [], "minkowski": []}
        for gallery_entry in gallery_embeddings:
            g_embedding = gallery_entry["embedding"]
            distances["euclidean"].append((euclidean_distance(probe_embedding, g_embedding), gallery_entry))
            distances["dot_product"].append((dot_product_distance(probe_embedding, g_embedding), gallery_entry))
            distances["cosine"].append((cosine_distance(probe_embedding, g_embedding), gallery_entry))
            distances["minkowski"].append((minkowski_distance(probe_embedding, g_embedding), gallery_entry))
        
        # Sort the distances for each metric (lower is better).
        for metric in distances:
            distances[metric].sort(key=lambda x: x[0])
        
        # Report the rank at which the correct gallery image is found.
        print(f"\nProbe Image: {person} (Transformation: {transform_name})")
        for metric, results_list in distances.items():
            rank_found = None
            for rank, (dist_val, gallery_entry) in enumerate(results_list, start=1):
                if gallery_entry["person"].lower() == person.lower():
                    rank_found = rank
                    break
            if rank_found is not None:
                print(f"  {metric} distance: Correct gallery image found at rank {rank_found}")
            else:
                print(f"  {metric} distance: No matching gallery image found in the gallery")
    print("-" * 60)



--- Rank Positions for Correct Gallery Matches on Transformed Probe Images ---


Transformation: horizontal_flip

Probe Image: Drew_Barrymore (Transformation: horizontal_flip)
  euclidean distance: Correct gallery image found at rank 188
  dot_product distance: Correct gallery image found at rank 188
  cosine distance: Correct gallery image found at rank 188
  minkowski distance: Correct gallery image found at rank 180

Probe Image: Warren_Buffett (Transformation: horizontal_flip)
  euclidean distance: Correct gallery image found at rank 113
  dot_product distance: Correct gallery image found at rank 113
  cosine distance: Correct gallery image found at rank 113
  minkowski distance: Correct gallery image found at rank 97

Probe Image: Owen_Wilson (Transformation: horizontal_flip)
  euclidean distance: Correct gallery image found at rank 117
  dot_product distance: Correct gallery image found at rank 117
  cosine distance: Correct gallery image found at rank 117
  minkowski distance: 

Here are some observations based on the results for Task 4:

- **Transformation Invariance Varies by Operation:**  
  - **Horizontal Flip:**  
    - For Drew_Barrymore, Warren_Buffett, and Owen_Wilson, the correct gallery images moved significantly higher in the ranking (e.g., Drew_Barrymore improved from rank 360 in the untransformed case to around rank 188).  
    - This suggests that the embedding model has some invariance to horizontal flips—perhaps even improving matching performance for certain identities.
  
  - **Gaussian Blur:**  
    - The performance with Gaussian blur appears similar to the untransformed scenario for many identities. For instance, Drew_Barrymore remains at rank 360, and Warren_Buffett at rank 393.  
    - This indicates that while moderate blur might not drastically distort the facial features, it does not offer the same benefit as a horizontal flip.

- **Sensitivity to Brightness Adjustments:**  
  - **Increase Brightness:**  
    - There is a dramatic degradation in performance for some individuals. Warren_Buffett’s rank jumps to 923, and Owen_Wilson’s rank escalates to over 2,190.  
    - Such high ranks suggest that increasing brightness severely disrupts the discriminative features captured by the embeddings, making it harder for the correct match to stand out.
  
  - **Decrease Brightness:**  
    - The effects are mixed. For Warren_Buffett, performance improves (rank 218) compared to the untransformed case, while for Owen_Wilson it only improves moderately (rank 414), and for Drew_Barrymore it remains similar.  
    - This variability indicates that the model’s robustness to reduced brightness depends on the specific facial features of each individual.

- **Robustness for Certain Identities:**  
  - Nelson_Mandela consistently achieves a rank of 1 across all transformations, suggesting that his facial features are highly distinctive and robust to various types of image alterations.
  - Ian_Thorpe shows moderate fluctuations (ranks in the 90s–100 range) across different transformations, hinting at some sensitivity but not as severe as the brightness increase effects observed for others.

- **Consistency Across Distance Metrics:**  
  - Across all transformations, the rank positions are nearly identical when computed with Euclidean, dot product, cosine, and Minkowski distances. This consistency reinforces that the relative structure of the embedding space is preserved regardless of the metric used.

Overall, these observations highlight that while some transformations (like horizontal flip) can improve retrieval performance for certain identities, others—especially extreme brightness adjustments—can severely impair matching. This suggests a need for careful calibration of preprocessing steps in real-world applications where image quality may vary.